In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import all dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
from glob import glob

from builtins import range, input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import confusion_matrix, roc_curve, classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

### Define Few Parameters

In [ ]:
# define size to which images are to be resized
IMAGE_SIZE = [224, 224]

# training config:
epochs = 500
batch_size = 32

# define paths
covid_path = '/content/drive/MyDrive/Colab Notebooks/Covid-19 CT Scan/data/CT_COVID'
noncovid_path = '/content/drive/MyDrive/Colab Notebooks/Covid-19 CT Scan/data/CT_NonCOVID'

# use gloab to grab images from path .jpg or .jpeg
covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

In [ ]:
# Visualize file variable contents

print('First 5 Covid Files : ', covid_files[0:5])
print('Total Count :', len(covid_files))

print('First 5 NonCovid Files : ', noncovid_files[0:5])
print('Total Count : ', len(noncovid_files))

### Fetch Images and Class Labels from Files (This might take a while)

In [ ]:
# fetch images and class labels from files

covid_labels = []
noncovid_labels = []

covid_images = []
noncovid_images = []

for i in range(len(covid_files)):
    image = cv2.imread(covid_files[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    covid_images.append(image)
    covid_labels.append('CT_COVID')
    
for i in range(len(noncovid_files)):
    image = cv2.imread(noncovid_files[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    noncovid_images.append(image)
    noncovid_labels.append('CT_NonCovID')

### Visualize First 40 Images from Data set

In [ ]:
# look at a random image for fun

def plot_images(images, title):
    nrows, ncols = 5, 8
    figsize = [10, 6]
    
    fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = figsize, facecolor = (1, 1, 1))
    
    for i, axi in enumerate(ax.flat):
        axi.imshow(images[i])
        axi.set_axis_off()
        
    plt.suptitle(title)
    plt.tight_layout(pad = 0.2, rect = [0, 0, 1, 0.9])
    plt.show()
    
plot_images(covid_images, 'Positive COVID-19 CT Scan')
plot_images(noncovid_images, 'Negative COVID-19 CT Scan')

### **Normalization**

Model takes images in the form of array of pixels. Hence convert into array and *normalize*

In [ ]:
# convert to array and normalize to interval of [0, 1]

covid_images = np.array(covid_images) / 255
noncovid_images = np.array(noncovid_images) / 255

### **Train Test Split**

In [ ]:
# Split into training and testing sets for both types of images

covid_x_train, covid_x_test, covid_y_train, covid_y_test = train_test_split(covid_images, covid_labels, test_size = 0.2)
noncovid_x_train, noncovid_x_test, noncovid_y_train, noncovid_y_test = train_test_split(noncovid_images, noncovid_labels, test_size = 0.2)

# Merge sets for both types of images

x_train = np.concatenate((noncovid_x_train, covid_x_train), axis = 0)
x_test = np.concatenate((noncovid_x_test, covid_x_test), axis = 0)
y_train = np.concatenate((noncovid_y_train, covid_y_train), axis = 0)
y_test = np.concatenate((noncovid_y_test, covid_y_test), axis = 0)

# Make labels into categories - either 0 or 1, for our model

y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)

### Visualize a few images from Training and Test sets

In [ ]:
plot_images(covid_x_train, 'x_train')
plot_images(covid_x_test, 'x_test')

# y_train and y_test contain class lables 0 and 1 representing COVID and NonCOVID for X_train and X_test

### **Building and Visualizing model**

In [ ]:
# building model

resnet = ResNet50(weights = 'imagenet', include_top = False, input_tensor = Input(shape = (224, 224, 3)))

outputs = resnet.output
outputs = Flatten(name = 'flatten')(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(2, activation = 'softmax')(outputs)

model = Model(inputs = resnet.input, outputs = outputs)

for layer in resnet.layers:
    layer.trainable = False
    
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# visualize Model

model.summary()

### **Image Augmentation**

To train on images at different positions, angles, flips, etc

In [ ]:
# image augmentation

image_aug = ImageDataGenerator(rotation_range = 20, 
                              width_shift_range = 0.2, 
                              height_shift_range = 0.2,
                              horizontal_flip = True)

### **Training the model**

In [ ]:
# training the model

history = model.fit(image_aug.flow(x_train, y_train, batch_size = batch_size), 
                   validation_data = (x_test, y_test),
                   validation_steps = len(x_test) / batch_size,
                   steps_per_epoch = len(x_train) / batch_size,
                   epochs = epochs)

### **Making Predicions**

In [ ]:
# save model and weights

model.save('/content/drive/MyDrive/Colab Notebooks/Covid-19 CT Scan/covid-19_ct_scan.h5')
model.save_weights('/content/drive/MyDrive/Colab Notebooks/Covid-19 CT Scan/covid-19_ct_scan_weights.h5')

In [ ]:
# load saved model

model = load_model('/content/drive/MyDrive/Colab Notebooks/Covid-19 CT Scan/covid-19_ct_scan.h5')

In [ ]:
y_pred = model.predict(x_test, batch_size = batch_size)

### Visulaizing First 10 predictions

In [ ]:
prediction = y_pred[0:10]

for index, probability in enumerate(prediction):
    if probability[1] > 0.5:
        plt.title('%.2f' % (probability[1] * 100) + '% COVID')
    else:
        plt.title('%.2f' % ((1 - probability[1]) * 100) + '% NonCOVID')
        
    plt.imshow(x_test[index])
    plt.show()

In [ ]:
# convert to binary classes

y_pred_bin = np.argmax(y_pred, axis = 1)
y_test_bin = np.argmax(y_test, axis = 1)

### Plot ROC Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_bin, y_pred_bin)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for our model')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

### Plot Confusion Matrix

In [ ]:
def plot_confusion_matrix(normalize):
  classes = ['COVID','NonCOVID']
  tick_marks = [0.5, 1.5]
  cn = confusion_matrix(y_test_bin, y_pred_bin, normalize = normalize)
  sns.heatmap(cn,cmap = 'viridis',annot=True)
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)
  plt.title('Confusion Matrix')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

print('Confusion Matrix without Normalization')
plot_confusion_matrix(normalize = None)

print('Confusion Matrix with Normalized Values')
plot_confusion_matrix(normalize = 'true')

### **Classification Report**

In [ ]:
print(classification_report(y_test_bin, y_pred_bin))

### **Accuracy and Loss Plots**

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])
plt.savefig('/content/drive/MyDrive/Colab Notebooks/Covid-19 CT Scan/covid-19_ct_scan_accuracy.png')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])
plt.savefig('/content/drive/MyDrive/Colab Notebooks/Covid-19 CT Scan/covid-19_ct_scan_loss.png')
plt.show()